In [7]:
my_str = "select * from sale.tbl_businessdoc;"

In [10]:
f'{my_str.rstrip(";")} LIMIT {100}'

'select * from sale.tbl_businessdoc LIMIT 100'

In [ ]:
import json

from openai import OpenAI
from sqlalchemy.orm import Session
from typing import List, Dict, Any, Tuple

from app.config import get_settings
from app.tools.sql_tool import SQLTool
from app.tools.schema_tool import SchemaTool
from app.tools.api_tools import ApiTool, CombinedDataTool
from app.schemas.chat import ToolCall
from app.core.prompts import get_contextual_prompt

ModuleNotFoundError: No module named 'app'

In [ ]:
settings = get_settings()

In [ ]:
class Llm_client:
    def __init__(self, ):
        self.client = OpenAI(api_key=settings.openai_api_key, base_url="")
        self.model = settings.openai_model

        self.tools = [
            # Data base tool
            SQLTool.get_tool_definition(),
            SchemaTool.get_tool_definition(),

            #api tool
            ApiTool.get_tool_definition(),
            CombinedDataTool.get_tool_definition()
        ]

    def _execute_tool(self, function_name: str, args: Dict[str, Any], db: Session):
        """Execute a tool function - support both data base and api tools
        """
        # ════════════════════════════════════════════════════════════════
        # DATA BASE TOOLS
        # ════════════════════════════════════════════════════════════════

        if function_name == "query_database":
            query = args.get("query", "")
            return SQLTool.execute_query(db, query)

        elif function_name == "get_schema":
            table_name = args.get("table_name", None)
            return SchemaTool.get_schema(db, table_name)

        # ════════════════════════════════════════════════════════════════
        # API TOOLS
        # ════════════════════════════════════════════════════════════════

        elif function_name == "call_backend_api":
            endpoint = args.get("endpoint", "")
            params = args.get("params", {})
            return ApiTool.call_backend_api(endpoint, params)

        elif function_name == "get_combined_data":
            data_needed = args.get("data_needed", "")
            vehicle_id = args.get("vehicle_id")

            results = {
                "database_data": {},
                "api_data": {},
                "combined": True
            }

            if vehicle_id:
                db_query = f"SELECT * FROM trips WHERE vehicle_id = '{vehicle_id}' ORDER BY trip_date DESC LIMIT 10"
                results["database_data"] = SQLTool.execute_query(db, db_query)

                # Get current location from API
                api_result = ApiTool.call_backend_api(
                    "/api/vehicles/location",
                    {"vehicle_id": vehicle_id}
                )
                results["api_data"] = api_result

            return results

        # ═══════════════════════════════════════════════
        # UNKNOWN TOOL
        # ═══════════════════════════════════════════════

        else:
            return {"error": f"Unknown tool: {function_name}"}

    def chat(self, user_message: str, conversation_history: List[Dict[str, str]], db: Session) -> Tuple[str, List[ToolCall]]:
        system_prompt = get_contextual_prompt(user_message)

        # Track and record chat history
        toolcalls_made = []
        messages = [{"role": "system", "content": system_prompt}]

        #check if conversation history is existed
        if conversation_history:
            messages.extend(conversation_history[: 1])

        messages.append({"role": "user", "content": user_message})

        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=self.tools,
            tool_choice="auto",
            max_tokens=500
        )

        # Create a Loop for iterative chat
        max_iteration = 5
        iteration = 0

        while iteration < max_iteration:
            iteration += 1
            assitant_message = response.choices[0].message
            messages.append(assitant_message)

            # check if there are tool calls
            if not assitant_message.tool_calls:
                # No more tool calls return final message
                return assitant_message.content, toolcalls_made

            # Track tools call (API's or DB) and Execute tools
            for tool_call in assitant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = tool_call.function.arguments

                result = self._execute_tools(function_name, function_args, db)
                toolcalls_made.append(ToolCall(tool_name=function_name, arguments=function_args, result=result))

                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "result": json.dumps(result)
                })

            # Get next response
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=self.tools,
                tool_choice="auto",
                max_tokens=500
            )

            return "متأسفانه تعداد مراحل از حد مجاز گذشت.", toolcalls_made